In [1]:
x = 0

In [2]:
import sys
import json
import numpy as np
import pandas as pd

print ("Importing modules...", end=' ')
import metal_prediction_modules
print ("Done")

##################################################

print ("Reading data from disk...", end=' ')
sys.stdout.flush()

df = pd.read_parquet('./datasets/Metal_all_20180601.parquet')
seqs = np.array(df.sequence)

print ("Done")

##################################################

print ("Using FOFE encoder...", end=' ')
sys.stdout.flush()
# FOFE
vocab_dic_fofe = {}
with open("./dictionaries/vocab_dict_fofe", 'r') as fp:
        vocab_dic_fofe = json.load(fp)
print ("Done")

Importing modules... 

Using TensorFlow backend.


Done
Reading data from disk... Done
Using FOFE encoder... Done


In [3]:
from Zihan.fofe import FofeVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import keras
import numpy as np

def FOFE(s): 

    ls = list(s)
    s2 = []
    for i in range(len(ls)-1):
        s2.append(ls[i]+ls[i+1])
    
    alpha = 0.95

    fofe = FofeVectorizer(alpha)
    features = fofe.naive_transform([s2], vocab_dic_fofe)
    features_rev = fofe.naive_transform([list(reversed(s2))], vocab_dic_fofe)

    # normalize feature matrix by subtracting each value by mean/std
    for i in range(len(features)):
        tmp_ls = np.array(features[i])
        tmp_std = np.std(tmp_ls, dtype=np.float32)
        tmp_mean = np.mean(tmp_ls, dtype=np.float32)
        for j in range(len(tmp_ls)):
            features[i][j]=tmp_ls[j]-tmp_mean/tmp_std

    # normalize feature matrix by subtracting each value by mean/std
    for i in range(len(features_rev)):
        tmp_ls = np.array(features_rev[i])
        tmp_std = np.std(tmp_ls, dtype=np.float32)
        tmp_mean = np.mean(tmp_ls, dtype=np.float32)
        for j in range(len(tmp_ls)):
            features_rev[i][j]=tmp_ls[j]-tmp_mean/tmp_std

    # Combine features and features_rev
    features_new = []
    for i in range(len(features)):
        features_new.append(np.concatenate((features[i],features_rev[i])))

    X = np.array(features_new)

    return X

In [4]:
# later...
from keras.models import model_from_json
# load json and create model
json_file = open('./models/metal_predict.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("./models/metal_predict.h5")
print("Loaded model from disk")

Loaded model from disk


In [5]:
model.predict(FOFE(seqs[x]))

array([[  5.97876846e-04,   1.53844003e-02,   1.01810216e-25,
          3.37811054e-15,   1.98241618e-10,   2.76620726e-09,
          7.61804283e-02,   9.07837272e-01]], dtype=float32)